In [1]:
import pandas as pd
import numpy as np
from datetime import datetime


In [2]:
testset=pd.read_csv("cleansed_testset.csv")
trainset=pd.read_csv("DA1920_train.csv")

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
store = pd.read_csv("cleansed_store.csv")

In [4]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenSinceDate,Promo2SinceDate
0,1,c,a,1270.0,9,2008,0,1,2016,0,2008-09-01,2016-01-10
1,2,a,a,570.0,11,2007,1,13,2010,1,2007-11-01,2010-04-04
2,3,a,a,14130.0,12,2006,1,14,2011,1,2006-12-01,2011-04-10
3,4,c,c,620.0,9,2009,0,1,2016,0,2009-09-01,2016-01-10
4,5,a,a,15770.0,4,2015,0,1,2016,0,2015-04-01,2016-01-10


In [5]:
train_merged = pd.merge(store,trainset,how="inner",on="Store")

In [6]:
train_merged.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenSinceDate,Promo2SinceDate,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,c,a,1270.0,9,2008,0,1,2016,0,2008-09-01,2016-01-10,5,31/07/2015,5263,555,1,1,0,1
1,1,c,a,1270.0,9,2008,0,1,2016,0,2008-09-01,2016-01-10,4,30/07/2015,5020,546,1,1,0,1
2,1,c,a,1270.0,9,2008,0,1,2016,0,2008-09-01,2016-01-10,3,29/07/2015,4782,523,1,1,0,1
3,1,c,a,1270.0,9,2008,0,1,2016,0,2008-09-01,2016-01-10,2,28/07/2015,5011,560,1,1,0,1
4,1,c,a,1270.0,9,2008,0,1,2016,0,2008-09-01,2016-01-10,1,27/07/2015,6102,612,1,1,0,1


In [7]:
test_merged = pd.merge(store,testset,how="inner",on="Store")

In [8]:
### Changing all datetime columns into the same datetime format

In [9]:
train_merged["Date"] = pd.to_datetime(train_merged["Date"].astype(str), format = '%d/%m/%Y')


In [10]:
train_merged["CompetitionOpenSinceDate"] = pd.to_datetime(train_merged["CompetitionOpenSinceDate"].astype(str), format = '%Y-%m-%d')
train_merged["Promo2SinceDate"] = pd.to_datetime(train_merged["Promo2SinceDate"].astype(str), format = '%Y-%m-%d')

In [11]:
### calculating days affected by competition 
### 

In [12]:
# calculating age of competitor in days
daysdiff_Date_COSD = train_merged["Date"]-train_merged["CompetitionOpenSinceDate"]
### chaninging timedelta dtype to integer
daysdiff_Date_COSD = daysdiff_Date_COSD.dt.days
daysdiff_Date_COSD = daysdiff_Date_COSD.to_numpy()


## replacing negative values (future openinng competitor) to 0

for i in range(len(daysdiff_Date_COSD)):
    if daysdiff_Date_COSD[i] < 0:
        daysdiff_Date_COSD[i] = 0


In [13]:
train_merged["competition_age_days"] = daysdiff_Date_COSD

In [14]:
##to use for loop, numpy is much faster so converting to numpy
np_Date = train_merged["Date"].to_numpy()
np_COSD = train_merged["CompetitionOpenSinceDate"].to_numpy()
#only promo2sincedate with store opened

np_P2SD = train_merged["Promo2SinceDate"].to_numpy()

In [15]:
#creating affectedbycompetition column first by putting in arbitary value
train_merged["affectedbycompetition"] = 3

In [16]:
## if sale date is more future than competitor open date, then there is impact of the competitor on the sales
for i in range(len(np_Date)):
    
    if np_Date[i] >= np_COSD[i]:
        train_merged["affectedbycompetition"][i] = 1
         
    else :
        train_merged["affectedbycompetition"][i] = 0
        

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
#Check
train_merged[train_merged["affectedbycompetition"]==0]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,competition_age_days,affectedbycompetition
3890,5,a,a,15770.0,4,2015,0,1,2016,0,...,2,2015-03-31,7692,659,1,1,0,0,0,0
3891,5,a,a,15770.0,4,2015,0,1,2016,0,...,1,2015-03-30,8847,770,1,1,0,0,0,0
3892,5,a,a,15770.0,4,2015,0,1,2016,0,...,7,2015-03-29,0,0,0,0,0,0,0,0
3893,5,a,a,15770.0,4,2015,0,1,2016,0,...,6,2015-03-28,2038,219,1,0,0,0,0,0
3894,5,a,a,15770.0,4,2015,0,1,2016,0,...,5,2015-03-27,4181,499,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013436,1111,a,a,1900.0,6,2014,1,31,2013,1,...,6,2013-01-05,3325,291,1,0,0,0,0,0
1013437,1111,a,a,1900.0,6,2014,1,31,2013,1,...,5,2013-01-04,4640,453,1,0,0,1,0,0
1013438,1111,a,a,1900.0,6,2014,1,31,2013,1,...,4,2013-01-03,4579,444,1,0,0,1,0,0
1013439,1111,a,a,1900.0,6,2014,1,31,2013,1,...,3,2013-01-02,5097,505,1,0,0,1,0,0


In [18]:
##creating affectedbypromo column by putting in arbitary value
train_merged["affectedbypromo2"] = 0

In [19]:
## if promo started earlier than the sales date, then there is promo impact on sales
for i in range(len(np_Date)):
    
    if np_Date[i] >= np_P2SD[i]:
        train_merged["affectedbypromo2"][i] = 1
         
    else :
        train_merged["affectedbypromo2"][i] = 0
        
    

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
#check
a = train_merged[train_merged["Promo2SinceDate"]!="2016-01-10"]
a[a["affectedbypromo2"]==0]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,competition_age_days,affectedbycompetition,affectedbypromo2
17926,20,d,a,2340.0,5,2009,1,40,2014,1,...,2014-06-30,9229,954,1,1,0,0,1886,1,0
17927,20,d,a,2340.0,5,2009,1,40,2014,1,...,2014-06-29,0,0,0,0,0,0,1885,1,0
17928,20,d,a,2340.0,5,2009,1,40,2014,1,...,2014-06-28,5572,566,1,0,0,0,1884,1,0
17929,20,d,a,2340.0,5,2009,1,40,2014,1,...,2014-06-27,7149,839,1,0,0,0,1883,1,0
17930,20,d,a,2340.0,5,2009,1,40,2014,1,...,2014-06-26,8118,997,1,0,0,0,1882,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013436,1111,a,a,1900.0,6,2014,1,31,2013,1,...,2013-01-05,3325,291,1,0,0,0,0,0,0
1013437,1111,a,a,1900.0,6,2014,1,31,2013,1,...,2013-01-04,4640,453,1,0,0,1,0,0,0
1013438,1111,a,a,1900.0,6,2014,1,31,2013,1,...,2013-01-03,4579,444,1,0,0,1,0,0,0
1013439,1111,a,a,1900.0,6,2014,1,31,2013,1,...,2013-01-02,5097,505,1,0,0,1,0,0,0


In [21]:
train_merged.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,competition_age_days,affectedbycompetition,affectedbypromo2
0,1,c,a,1270.0,9,2008,0,1,2016,0,...,2015-07-31,5263,555,1,1,0,1,2524,1,0
1,1,c,a,1270.0,9,2008,0,1,2016,0,...,2015-07-30,5020,546,1,1,0,1,2523,1,0
2,1,c,a,1270.0,9,2008,0,1,2016,0,...,2015-07-29,4782,523,1,1,0,1,2522,1,0
3,1,c,a,1270.0,9,2008,0,1,2016,0,...,2015-07-28,5011,560,1,1,0,1,2521,1,0
4,1,c,a,1270.0,9,2008,0,1,2016,0,...,2015-07-27,6102,612,1,1,0,1,2520,1,0


In [22]:
## since i got "affectedbycompetition" & "affectedbypromo"columns, now I delete promo2sincedate and competitionopensince date columns
#del train_merged["Promo2SinceDate"]
#del train_merged["CompetitionOpenSinceDate"]

In [23]:
## Repeat same process to test_merged. First, change each date column to datetime dtype

In [24]:
test_merged["Date"] = pd.to_datetime(test_merged["Date"].astype(str), format = '%d/%m/%Y')
test_merged["CompetitionOpenSinceDate"] = pd.to_datetime(test_merged["CompetitionOpenSinceDate"].astype(str), format = '%Y-%m-%d')
test_merged["Promo2SinceDate"] = pd.to_datetime(test_merged["Promo2SinceDate"].astype(str), format = '%Y-%m-%d')

In [25]:
## To use forloop, convert to np (faster)

In [26]:
np_P2SD2 = test_merged["Promo2SinceDate"].to_numpy()
np_Date2 = test_merged["Date"].to_numpy()
np_COSD2 = test_merged["CompetitionOpenSinceDate"].to_numpy()

In [27]:
# creating a new column affectedbypromo2

In [28]:
test_merged["affectedbypromo2"] = 33

In [29]:
## if promo2 started earlier than the sale date,  sale is affected by promo2

In [30]:
for i in range(len(np_Date2)):
    
    if np_Date2[i] >= np_P2SD2[i]:
        test_merged["affectedbypromo2"][i] = 1
         
    else :
        test_merged["affectedbypromo2"][i] = 0

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
#creating affectedbycompetition column

In [32]:
test_merged["affectedbycompetition"] = 33

In [33]:
# if competition open date was earlier than sales date, sales is affected by competition

In [34]:

for i in range(len(np_Date2)):
    
    if np_Date2[i] >= np_COSD2[i]:
        test_merged["affectedbycompetition"][i] = 1
         
    else :
        test_merged["affectedbycompetition"][i] = 0

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [35]:
## Creating age of competitor column

In [36]:
# calculating age of competitor in days
daysdiff_Date_COSD2 = test_merged["Date"]-test_merged["CompetitionOpenSinceDate"]
### chaninging timedelta dtype to integer
daysdiff_Date_COSD2 = daysdiff_Date_COSD2.dt.days
daysdiff_Date_COSD2 = daysdiff_Date_COSD2.to_numpy()


## replacing negative values (future openinng competitor) to 0

for i in range(len(daysdiff_Date_COSD2)):
    if daysdiff_Date_COSD2[i] < 0:
        daysdiff_Date_COSD2[i] = 0

test_merged["competition_age_days"] = daysdiff_Date_COSD2

In [37]:
## now delete date columns

In [38]:
#del test_merged["CompetitionOpenSinceDate"]
#del test_merged["Promo2SinceDate"]

In [39]:
train_merged[train_merged["competition_age_days"]==0]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,competition_age_days,affectedbycompetition,affectedbypromo2
3889,5,a,a,15770.0,4,2015,0,1,2016,0,...,2015-04-01,7720,773,1,1,0,0,0,1,0
3890,5,a,a,15770.0,4,2015,0,1,2016,0,...,2015-03-31,7692,659,1,1,0,0,0,0,0
3891,5,a,a,15770.0,4,2015,0,1,2016,0,...,2015-03-30,8847,770,1,1,0,0,0,0,0
3892,5,a,a,15770.0,4,2015,0,1,2016,0,...,2015-03-29,0,0,0,0,0,0,0,0,0
3893,5,a,a,15770.0,4,2015,0,1,2016,0,...,2015-03-28,2038,219,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013436,1111,a,a,1900.0,6,2014,1,31,2013,1,...,2013-01-05,3325,291,1,0,0,0,0,0,0
1013437,1111,a,a,1900.0,6,2014,1,31,2013,1,...,2013-01-04,4640,453,1,0,0,1,0,0,0
1013438,1111,a,a,1900.0,6,2014,1,31,2013,1,...,2013-01-03,4579,444,1,0,0,1,0,0,0
1013439,1111,a,a,1900.0,6,2014,1,31,2013,1,...,2013-01-02,5097,505,1,0,0,1,0,0,0


In [40]:
## changing StateHoliday, StoreType, Assortment letters into categorical number
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train_merged["StateHoliday"] = train_merged["StateHoliday"].apply(str)
train_merged["StateHoliday"] = le.fit_transform(train_merged["StateHoliday"])
train_merged["StoreType"] = le.fit_transform(train_merged["StoreType"])
train_merged["Assortment"] = le.fit_transform(train_merged["Assortment"])

test_merged["StateHoliday"] = test_merged["StateHoliday"].apply(str)
test_merged["StateHoliday"] = le.fit_transform(test_merged["StateHoliday"])
test_merged["StoreType"] = le.fit_transform(test_merged["StoreType"])
test_merged["Assortment"] = le.fit_transform(test_merged["Assortment"])

In [41]:
train_merged.to_csv("train_merged.csv",index=False)
test_merged.to_csv("test_merged.csv",index=False)

In [42]:
train_merged.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,competition_age_days,affectedbycompetition,affectedbypromo2
0,1,2,0,1270.0,9,2008,0,1,2016,0,...,2015-07-31,5263,555,1,1,0,1,2524,1,0
1,1,2,0,1270.0,9,2008,0,1,2016,0,...,2015-07-30,5020,546,1,1,0,1,2523,1,0
2,1,2,0,1270.0,9,2008,0,1,2016,0,...,2015-07-29,4782,523,1,1,0,1,2522,1,0
3,1,2,0,1270.0,9,2008,0,1,2016,0,...,2015-07-28,5011,560,1,1,0,1,2521,1,0
4,1,2,0,1270.0,9,2008,0,1,2016,0,...,2015-07-27,6102,612,1,1,0,1,2520,1,0
